In [52]:
import numpy as np
import matplotlib.pyplot as plt
import preprocessing as pp
import implementations as imp
import helpers as hlp
import os
from functions import *
import index as idx 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data cleaning and Preprocessing

In [18]:
path = "/Users/maelynenguyen/Desktop/"

x_train_ = np.load(path+"f_x_train_.npy")
x_test_ = np.load(path+"f_x_test_.npy")
y_train_ = np.load(path+"f_y_train_.npy")
test_ids = np.load(path+"f_test_ids_.npy")
train_ids = np.load(path+"f_train_ids_.npy")

In [51]:
path_ = '/Users/maelynenguyen/Desktop/dataset_to_release'

x_train_t, x_test_t, y_train_t, train_ids_, test_ids_ = hlp.load_csv_data(path_)

In [53]:
import csv

# Load the CSV files and extract the feature names
with open('/Users/maelynenguyen/Desktop/dataset_to_release/x_train.csv', 'r') as f:
    reader = csv.reader(f)
    x_train_features = next(reader)

with open('/Users/maelynenguyen/Desktop/dataset_to_release/x_test.csv', 'r') as f:
    reader = csv.reader(f)
    x_test_features = next(reader)

print("Features in x_train.csv:", x_train_features)
print("Features in x_test.csv:", x_test_features)

Features in x_train.csv: ['Id', '_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'DISPCODE', 'SEQNO', '_PSU', 'CTELENUM', 'PVTRESD1', 'COLGHOUS', 'STATERES', 'CELLFON3', 'LADULT', 'NUMADULT', 'NUMMEN', 'NUMWOMEN', 'CTELNUM1', 'CELLFON2', 'CADULT', 'PVTRESD2', 'CCLGHOUS', 'CSTATE', 'LANDLINE', 'HHADULT', 'GENHLTH', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'HLTHPLN1', 'PERSDOC2', 'MEDCOST', 'CHECKUP1', 'BPHIGH4', 'BPMEDS', 'BLOODCHO', 'CHOLCHK', 'TOLDHI2', 'CVDSTRK3', 'ASTHMA3', 'ASTHNOW', 'CHCSCNCR', 'CHCOCNCR', 'CHCCOPD1', 'HAVARTH3', 'ADDEPEV2', 'CHCKIDNY', 'DIABETE3', 'DIABAGE2', 'SEX', 'MARITAL', 'EDUCA', 'RENTHOM1', 'NUMHHOL2', 'NUMPHON2', 'CPDEMO1', 'VETERAN3', 'EMPLOY1', 'CHILDREN', 'INCOME2', 'INTERNET', 'WEIGHT2', 'HEIGHT3', 'PREGNANT', 'QLACTLM2', 'USEEQUIP', 'BLIND', 'DECIDE', 'DIFFWALK', 'DIFFDRES', 'DIFFALON', 'SMOKE100', 'SMOKDAY2', 'STOPSMK2', 'LASTSMK2', 'USENOW3', 'ALCDAY5', 'AVEDRNK2', 'DRNK3GE5', 'MAXDRNKS', 'FRUITJU1', 'FRUIT1', 'FVBEANS', 'FVGREEN', 'FVORANG', 'VE

In [55]:
index_9 = idx.index_remplace_9(x_train_features)
index_7 = idx.index_remplace_7(x_train_features)

In [ ]:
x_train_, x_test_ = idx.remplace(index_9,x_train_,x_test_)
x_train_, x_test_ = idx.remplace(index_7,x_train_,x_test_)

In [58]:
x, x_submit, y = pp.clean_data_final(x_train_,  y_train_t, x_test_)

# important to add a constant term for the bias
x_train = np.concatenate((x,np.zeros((x.shape[0],1))+1),axis=1)
x_test = np.concatenate((x_submit,np.zeros((x_submit.shape[0],1))+1),axis=1)


assert x.shape[1]+1 == x_train.shape[1]

y_train = y.copy()
y_train = np.where(y == -1, 0, 1)  # change -1 to 0
y_train = y_train.astype(int)      # change to int


Max median NaN score rows :  0.6060963867246555
Max median NaN score columns :  0.7652518774030354
Number of rows dropped because of a NaN score > 0.5:  37081
Number of columns dropped because of a NaN score > 0.5:  150
Number of columns with std < 0.1: 5
Number of columns with correl_coef > 0.95: 11
Handling NaN values...
Data clipped between 5th and 95th percentiles
Number of columns with std < 0.1 after cleaning: 11
Number of columns with corr_coef> 0.95 after cleaning: 14
The data has been cleaned and standardized
The cleaned x-data has the following shape:  (291054, 130)
The cleaned y-data has the following shape:  (291054,)
The cleaned x-data-to-predict has the following shape:  (109379, 130)


## Logistic Regression

### Test for specific hyperparameters

In [35]:
w, loss, losses = imp.logistic_regression(y_train, x_train, np.zeros(x_train.shape[1]), 1500, 0.1, True)

In [36]:
y_pred = imp.sigmoid(x_train@w)
thr = best_threshold(y_pred,y) # y is (-1,1) and y_pred is (0,1)
y_pred_ = np.where(y_pred > thr, 1, -1) # y_pred_ is (-1,1)

In [37]:
f1_score_(y, y_pred_)

0.41700883230437485

### Implementing a Cross Validation in a Grid Search to find the best parameters

In [38]:
param_grid = {
    "max_iters": [1500],
    "gamma": [1,0.1, 0.001],
    "lambda_": [1e-4,1e-5,1e-6],
}

In [39]:
w_grid, best_params, losses = grid_search_logistic_regression(y_train, x_train, param_grid, np.zeros(x_train.shape[1]))


Max Iters: 1500, Gamma: 1, Avg Loss: 0.22977808706011765
Max Iters: 1500, Gamma: 0.1, Avg Loss: 0.22991349322823157
Max Iters: 1500, Gamma: 0.001, Avg Loss: 0.4900182968043444


In [40]:
print("Best hyperparameters found: ", best_params)

Best hyperparameters found:  {'max_iters': 1500, 'gamma': 1}


In [41]:
y_train_pred = imp.sigmoid(x_train@w_grid) # y_train_pred is (0,1)
optimal_threshold = best_threshold(y_train_pred,y) 
print("Optimal threshold found: ", optimal_threshold)


Optimal threshold found:  0.19


In [42]:
y_pred = imp.sigmoid(x_test@w_grid)
y_pred_ = np.where(y_pred > optimal_threshold, 1, -1)
hlp.create_csv_submission(test_ids_, y_pred_, "logistic_regression24102147.csv")

### Testing of an Adam optimizer 

In [59]:
w_adam = adam_optimizer(y_train, x_train, np.zeros(x_train.shape[1]), 1500, 0.1)

y_train_adam = imp.sigmoid(np.dot(x_train, w_adam)) 
optimal_threshold = best_threshold(y_train_adam, y) # y_train is (-1,1) and y_train_adam is (0,1)

y_pred_adam = imp.sigmoid(np.dot(x_test, w_adam))
y_pred_adam  = np.where(y_pred_adam > optimal_threshold , 1, -1)

hlp.create_csv_submission(test_ids_, y_pred_adam, "adam_final.csv")

### Logistic Regression with Regularization 

In [43]:
def k_fold_split(x, y, k):
    """Utility function to split data into k folds."""
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    fold_size = len(y) // k
    folds = []
    
    for i in range(k):
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.concatenate((indices[:i * fold_size], indices[(i + 1) * fold_size:]))
        folds.append((train_indices, test_indices))
    
    return folds

In [44]:
def grid_search_reg_logistic_regression(y_train, x_train_cleaned, param_grid, w_initial, k=5):
    best_params = None
    best_score = float("inf")
    best_w = w_initial
    losses = []
        
    # Generate folds for cross-validation
    folds = k_fold_split(x_train_cleaned, y_train, k)

    # Iterate over each combination of parameters
    for max_iters in param_grid["max_iters"]:
        for gamma in param_grid["gamma"]:
            for lambda_ in param_grid["lambda_"]:
                total_loss = 0    
                # Perform k-fold cross-validation
                for train_indices, test_indices in folds:
                    x_train_fold = x_train_cleaned[train_indices]
                    y_train_fold = y_train[train_indices]
                    x_test_fold = x_train_cleaned[test_indices]
                    y_test_fold = y_train[test_indices]
                    
                    # Train the model with training data
                    w, _ = imp.reg_logistic_regression(y_train_fold, x_train_fold, lambda_, w_initial, max_iters, gamma)
                    # Test the model with testing data
                    loss = imp.logistic_loss(y_test_fold, x_test_fold, w)
                    
                    total_loss += loss

                avg_loss = total_loss / k
                losses.append((max_iters, gamma, lambda_, avg_loss))
                print(f"Max Iters: {max_iters}, Gamma: {gamma}, Lambda: {lambda_}, Avg Loss: {avg_loss}")

                if avg_loss < best_score:
                    best_score = avg_loss
                    best_w = w
                    best_params = {
                        "max_iters": max_iters,
                        "gamma": gamma,
                        "lambda_": lambda_,
                    }

    return best_w, best_params, losses


In [45]:
w_reg, best_params, losses = grid_search_reg_logistic_regression(y_train, x_train, param_grid, np.zeros(x_train.shape[1]))
#w_reg, loss_reg = imp.reg_logistic_regression(y_train, x_train, 1e-6, np.zeros(x_train.shape[1]), 1500, 0.1)

Max Iters: 1500, Gamma: 1, Lambda: 0.0001, Avg Loss: 0.2297606158482602
Max Iters: 1500, Gamma: 1, Lambda: 1e-05, Avg Loss: 0.22976041756546514
Max Iters: 1500, Gamma: 1, Lambda: 1e-06, Avg Loss: 0.2297616920874618
Max Iters: 1500, Gamma: 0.1, Lambda: 0.0001, Avg Loss: 0.22993585946287043
Max Iters: 1500, Gamma: 0.1, Lambda: 1e-05, Avg Loss: 0.2298919415847535
Max Iters: 1500, Gamma: 0.1, Lambda: 1e-06, Avg Loss: 0.22988811346281293
Max Iters: 1500, Gamma: 0.001, Lambda: 0.0001, Avg Loss: 0.4900305808722318
Max Iters: 1500, Gamma: 0.001, Lambda: 1e-05, Avg Loss: 0.4900109562359458
Max Iters: 1500, Gamma: 0.001, Lambda: 1e-06, Avg Loss: 0.49000899364689304


In [46]:
print("Best hyperparameters found: ", best_params)

Best hyperparameters found:  {'max_iters': 1500, 'gamma': 1, 'lambda_': 1e-05}


In [47]:
y_train_reg = imp.sigmoid(np.dot(x_train, w_reg))
optimal_threshold_reg = best_threshold(y_train_reg, y) # y_train is (-1,1) and y_train_reg is (0,1)
print("Optimal threshold found for the regularized model: ", optimal_threshold_reg)

Optimal threshold found for the regularized model:  0.19


In [48]:
y_pred_reg = imp.sigmoid(np.dot(x_test, w_reg))
y_pred_reg = np.where(y_pred_reg > optimal_threshold_reg, 1, -1)

hlp.create_csv_submission(test_ids_, y_pred_reg, "y_pred_reg27101206.csv")